Required Packages

In [ ]:
# numpy==1.22.3
# pandas==1.4.2
# PyMsgBox==1.0.9
# python-dateutil==2.8.2
# pytz==2022.1
# pywin32==303
# pyxlsb==1.0.9
# six==1.16.0
# tk==0.1.0
# xlwings==0.27.5

In [ ]:
#import required modules
import os
import asyncio
import functools
import pandas as pd

from cluster.cluster import Cluster
from concurrent.futures import ThreadPoolExecutor
from dataframes.dataframe_optimized import DataFrameOptimized
from utils import constants as const, index as utils, feature_flags as ft

load files

In [ ]:
results = app.results
sources = config["sources"]

bases = {}

with ThreadPoolExecutor(max_workers=4) as executor:

    futures = []
    keys = []

    for key, source in sources.items():
        path = results[key].split("|") #list[base, ...] - key is a name of base
        if len(path) == 1:
            path = path[0]
        keys.append(key)
        futures.append(loop.run_in_executor(executor, functools.partial(Cluster.preprocess_base, **{"path": path, "properties": source})))

    results = await asyncio.gather(*futures)
        

bases = {f"{file.split('.')[0]}": DataFrameOptimized(pd.read_csv(os.path.join(const.ROOT_DIR, f"files/temp/{file}"))) for file in os.listdir(os.path.join(const.ROOT_DIR, f"files/temp"))}
# bases = {f"{keys[idx]}": result for idx, result in enumerate(results)}

final_base = Cluster()
await final_base.merge_all(bases, config["order_base"])

app.prog_bars["pb_process"].stop()
app.update_label(label="lbl_status", label_text="status_project", text="Terminado") # status